<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/GloVe_TweetsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip - ‘glove.6B.zip’ saved [862182613/862182613]

--2021-04-09 17:01:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-09 17:01:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-09 17:01:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall('data')
zip_ref.close()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [4]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)
              
    
def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))
        
        
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)
    
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    
    
def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [5]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [6]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import re
import requests

In [7]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/test_labels.txt"

In [8]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

114435

In [9]:
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()

val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()

test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

In [10]:
tweets[7]

'More #tinyepic things #tinyepicwestern, this one is crazy @user I may be one of your… \n'

In [11]:
for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    tweets[i]=re.sub('@user', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])

In [12]:
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()

val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()

test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()

In [13]:
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
# labels    
# Words like effing converted to VetsResistSquadron

val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))

test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))

In [14]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [15]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:

# def clean_tweets(tweet, stemmer=PorterStemmer(), stop_words=set(stopwords.words('english'))):
#   words=word_tokenize(tweet.lower())
#   filtered_words=[]
#   # for word in words:
#   #   if word not in stop_words and word.isalpha():
#   #     stemmed_word=stemmer.stem(word)
#   #     filtered_words.append(stemmed_word)
#   for word in words:
#     if word.isalpha():
#       # stemmed_word=stemmer.stem(word)
#       filtered_words.append(word)
#   return filtered_words

In [17]:
tweetsList=df.TWEET
# nestedList=tweetsList.apply(clean_tweets)
# nestedList[0:5]
val_tweetsList=val_df.TWEET
# val_nestedList=val_tweetsList.apply(clean_tweets)
# val_nestedList[0:5]
test_tweetsList=test_df.TWEET
# test_nestedList=test_tweetsList.apply(clean_tweets)
# test_nestedList[0:5]

In [18]:
np.asarray(tweetsList)

array(['Sunday afternoon walking through Venice in the sun with  ️ ️ ️ @ Abbot Kinney, Venice \n',
       "Time for some BBQ and whiskey libations. Chomp, belch, chomp! (@ Lucille's Smokehouse Bar-B-Que) \n",
       'Love love love all these people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ San… \n',
       ..., 'Be safe this weekend everyone. #happylaborday @ Beer NV \n',
       'Pizza (@ Five50 -  in Las Vegas, NV) \n',
       'my mini is perfect, no one deserves her @ Las Vegas, Nevada \n'],
      dtype=object)

In [19]:
# nestedList=tuple(nestedList)
# val_nestedList=tuple(val_nestedList)
# test_nestedList=tuple(test_nestedList)

In [20]:
# uniquewords_train=set()
# for tw in nestedList:
#   for w in tw:
#     uniquewords_train.add(w)

In [21]:
# uniquewords_train=tuple(uniquewords_train)
# word_id={}
# id_word={}
# i=1
# for word in uniquewords_train:
#   word_id[word]=i
#   i+=1

# for word, id in word_id.items():
#   id_word[id]=word

In [22]:
# X_train=[]
# twt=[]
# for tw in nestedList:
#   for word in tw:
#     wordid=word_id[word]
#     twt.append(wordid)
#   X_train.append(twt)
#   twt=[]

# twt=[]
# X_val=[]
# for tw in val_nestedList:
#   for word in tw:
#     wordid=word_id.get(word, 0)
#     twt.append(wordid)
#   X_val.append(twt)
#   twt=[]

# twt=[]
# X_test=[]
# for tw in test_nestedList:
#   for word in tw:
#     wordid=word_id.get(word, 0)
#     twt.append(wordid)
#   X_test.append(twt)
#   twt=[]

In [25]:
y_train=df.CATEGORY
y_val=val_df.CATEGORY
y_test=test_df.CATEGORY

In [26]:
# X_train, Y_train = read_csv('data/train_emoji.csv')
# X_test, Y_test = read_csv('data/tesss.csv')
X_train=np.array(df.TWEET)
X_test=np.asarray(test_df.TWEET)
Y_train=np.asarray(y_train)
Y_test=np.asarray(y_test)

In [76]:
X_train=X_train.squeeze()
X_test=X_test.squeeze()


In [77]:
maxLen = len(max(X_train, key=len).split())
maxLen

29

In [87]:
X_train.shape

(45000,)

In [106]:
def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]                                   # number of training examples
    
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        sentence_words =X[i].lower().split()
        
        j = 0
        
        for w in sentence_words:
          if j<max_len:
            X_indices[i, j] = word_to_index.get(w, 0)
            j = j + 1
            
    
    return X_indices

In [107]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [108]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    
    sentence_indices = Input(input_shape, dtype='int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)    
    
    X = LSTM(128, return_sequences=True)(embeddings)
    
    X = Dropout(0.5)(X)
    
    X = LSTM(128, return_sequences=False)(X)
    
    X = Dropout(0.5)(X)
    
    X = Dense(20)(X)
    
    X = Activation('softmax')(X)
    
    model = Model(inputs=sentence_indices, outputs=X)
      
    return model

In [109]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 29)]              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 29, 50)            20000050  
_________________________________________________________________
lstm_22 (LSTM)               (None, 29, 128)           91648     
_________________________________________________________________
dropout_22 (Dropout)         (None, 29, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_23 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 20)                258

In [110]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [111]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 20)

In [113]:
model.fit(X_train_indices, Y_train_oh, epochs = 150, batch_size = 32, shuffle=True)

Epoch 1/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2821 - accuracy: 0.6054
Epoch 2/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2754 - accuracy: 0.6074
Epoch 3/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2587 - accuracy: 0.6127
Epoch 4/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2433 - accuracy: 0.6148
Epoch 5/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2404 - accuracy: 0.6166
Epoch 6/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2272 - accuracy: 0.6217
Epoch 7/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2199 - accuracy: 0.6238
Epoch 8/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2091 - accuracy: 0.6272
Epoch 9/150
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1963 - accuracy: 0.6270
Epoch 10/150
1407/1407 [==============================] - 11s 8m

In [114]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 20)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

1563/1563 [==============================] - 7s 4ms/step - loss: 5.1711 - accuracy: 0.2149

Test accuracy =  0.21493999660015106
